In [1]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
import time

In [2]:
%%time
# Fetch dataset using sklearn
cov = fetch_covtype()
X = cov.data
y = cov.target

CPU times: user 660 ms, sys: 68 ms, total: 728 ms
Wall time: 724 ms


In [3]:
%%time
# Create 0.75/0.25 train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75,
                                                    random_state=42)

CPU times: user 108 ms, sys: 76 ms, total: 184 ms
Wall time: 182 ms


In [5]:
# Specify sufficient boosting iterations to reach a minimum
num_round = 10

# Leave most parameters as default
param = {'objective': 'multi:softmax', # Specify multiclass classification
         'num_class': 8, # Number of possible output classes
         'tree_method': 'gpu_hist' # Use GPU accelerated algorithm
         }

In [6]:
%%time
# Convert input data from numpy to XGBoost format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

CPU times: user 148 ms, sys: 88 ms, total: 236 ms
Wall time: 235 ms


In [7]:
%%time
gpu_res = {} # Store accuracy result
tmp = time.time()
# Train model
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_res)
print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))

[0]	test-merror:0.254804
[1]	test-merror:0.247892
[2]	test-merror:0.244263
[3]	test-merror:0.240683
[4]	test-merror:0.23848
[5]	test-merror:0.234777
[6]	test-merror:0.232154
[7]	test-merror:0.22973
[8]	test-merror:0.227169
[9]	test-merror:0.224629
GPU Training Time: 6.101988077163696 seconds
CPU times: user 35.7 s, sys: 1.09 s, total: 36.8 s
Wall time: 6.1 s


In [8]:
%%time
# Repeat for CPU algorithm
tmp = time.time()
param['tree_method'] = 'hist'
cpu_res = {}
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_res)
print("CPU Training Time: %s seconds" % (str(time.time() - tmp)))

[0]	test-merror:0.254831
[1]	test-merror:0.247912
[2]	test-merror:0.244298
[3]	test-merror:0.24069
[4]	test-merror:0.238536
[5]	test-merror:0.234804
[6]	test-merror:0.232229
[7]	test-merror:0.229703
[8]	test-merror:0.227162
[9]	test-merror:0.224519
CPU Training Time: 15.392259359359741 seconds
CPU times: user 6min 3s, sys: 904 ms, total: 6min 4s
Wall time: 15.4 s
